In [1]:
# importing labraries
import pandas as pd

# Reading all the files 
categories = pd.read_csv('data/categories.csv')
cities = pd.read_csv('data/cities.csv')
countries = pd.read_csv('data/countries.csv')
employees = pd.read_csv('data/employees.csv')
products = pd.read_csv('data/products.csv')
customers = pd.read_csv('data/customers.csv')
sales = pd.read_csv('data/sales.csv')


In [2]:
# Checking the structure of each files
for name, df in [('categories', categories), ('cities', cities), ('countries', countries), ('employees', employees), ('products', products),('customers', customers), ('sales', sales)]:
    print(f"\n{name.upper()}")
    print(df.info())
    print(df.head(3))




CATEGORIES
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CategoryID    11 non-null     int64 
 1   CategoryName  11 non-null     object
dtypes: int64(1), object(1)
memory usage: 308.0+ bytes
None
   CategoryID CategoryName
0           1  Confections
1           2   Shell fish
2           3      Cereals

CITIES
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CityID     96 non-null     int64 
 1   CityName   96 non-null     object
 2   Zipcode    96 non-null     int64 
 3   CountryID  96 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 3.1+ KB
None
   CityID CityName  Zipcode  CountryID
0       1   Dayton    80563         32
1       2  Buffalo    17420         32
2       3  Chicago    44751       

In [ ]:
import duckdb
# create a connection 
con = duckdb.connect()

# merge datasets directly from CSVs
merged_df = con.execute("""
    SELECT s.*, p.ProductName, p.CategoryID, c.CustomerName, c.Country
    FROM 'data/sales.csv' AS s
    LEFT JOIN 'data/products.csv' AS p ON s.ProductID = p.ProductID
    LEFT JOIN 'data/customers.csv' AS c ON s.CustomerID = p.CustomerID                        
""").df()

print(merged_df.head())

BinderException: Binder Error: Table "p" does not have a column named "CustomerID"

Candidate bindings: : "CategoryID"

LINE 5:     LEFT JOIN 'data/customers.csv' AS c ON s.CustomerID = p.CustomerID                        
                                                                  ^

In [ ]:
# Merging sales + products + customers + employees
chunk_size = 100000
chunks = []
for chunk in pd.read_csv('data/sales.csv', chunksize=chunk_size):
    merged = sales.merge(products, on='ProductID', how='left') \
        .merge(customers, on='CustomerID', how='left')
    
    chunks.append(merged)

merged_df = pd.concat(chunks, ignore_index=t)
print(merged_df.head())
print(merged_df.shape)